In [1]:
%load_ext autoreload
%autoreload 2

In [69]:
import os
import email as em

from datetime import date,timedelta,datetime
from time import mktime
from email.utils import parsedate, parsedate_tz
from email.parser import HeaderParser

from classes.gmail import GmailAccount

In [70]:
password = open(os.path.expanduser('~/.ssh/pw'), 'r').read().strip()

In [71]:
tyler = GmailAccount(username='tyleha@gmail.com', password=password)
tyler.login()

('OK', [b'tyleha@gmail.com authenticated (Success)'])

In [72]:
daysback = 23
notsince = 0
since = (date.today() - timedelta(daysback)).strftime("%d-%b-%Y")
before = (date.today() - timedelta(notsince)).strftime("%d-%b-%Y")

SEARCH = '(SENTSINCE {si} SENTBEFORE {bf})'.format(si=since, bf=before)
BODY = '(BODY.PEEK[TEXT])'
ALL_HEADERS = '(BODY.PEEK[HEADER.FIELDS (DATE TO CC FROM SUBJECT)])'
DATE = '(BODY.PEEK[HEADER.FIELDS (DATE)])'

In [98]:
#LOAD GMAIL EMAILS
received = tyler.load_parse_query(SEARCH, ALL_HEADERS, 'INBOX')
# sent = tyler.load_parse_query(SEARCH, ALL_HEADERS, '[Gmail]/Sent Mail')

# Do the pandas thing

In [99]:
import pandas as pd

In [100]:
df = pd.DataFrame([dict(email._headers) for email in received])

In [118]:
df['epoch_date'] = df['Date'].apply(lambda x: mktime(parsedate(x)))
df['day'] = df['epoch_date'].apply(lambda x: datetime.fromtimestamp(x).date())
df['time'] = df['epoch_date'].apply(lambda x: datetime.fromtimestamp(x).time())

In [102]:
datetime.fromtimestamp(received[0]['Date'])

TypeError: an integer is required (got type str)

In [104]:
date = received[0]['Date']

In [111]:
t = mktime(parsedate(date))
start = datetime.fromtimestamp(t)

In [112]:
start

datetime.datetime(2015, 12, 8, 3, 18, 14)

In [114]:
start.time()

datetime.time(3, 18, 14)

In [116]:
start.date()

datetime.date(2015, 12, 8)